In [13]:
word_to_number = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5
}

import numpy as np
import random
import torch

alpha = 0.10
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

import json
import math
import pandas as pd
from matplotlib import pyplot as plt
import os, sys
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')
sys.path.insert(0, os.path.abspath("../boosted-conformal/"))
third_party_path = os.path.abspath("../boosted-conformal/third_party")
sys.path.insert(0, third_party_path)

from boostedCP.utils import cqr_preboost,local_preboost,plot_len
from boostedCP.gradient_boost import gradient_boost_len
from boostedCP.len_local_boost import len_local_boost
from boostedCP.len_cqr_boost import len_cqr_boost

In [14]:
import time
import tracemalloc

In [15]:
import json
import pandas as pd
import numpy as np
import random
import time

def range_modification(y_qlow, y_qup, range_low,  range_up):
    y_qlow = np.clip(y_qlow, range_low, range_up)
    y_qup = np.clip(y_qup, range_low, range_up)
    return y_qlow, y_qup

def run_experiment(X, y, seed, type = 'cqr', dataset = 'summeval', dimension = 'consistency'):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    X = X.to_numpy().astype(np.float32)
    y = y.to_numpy().astype(np.float32).ravel()

    from sklearn.model_selection import train_test_split
    X_cal, X_test, y_cal, y_test = train_test_split(X, y, test_size=0.5, random_state=seed)

    if type == 'lcp':
        results = len_local_boost(X_cal, y_cal, X_cal, y_cal,X_test, y_test, 
                                alpha, seed, n_rounds_cv=500, learning_rate=0.02, store=True, verbose=False)
    if type == 'cqr':
        results = len_cqr_boost(X_cal, y_cal, X_cal, y_cal,X_test, y_test, 
                                alpha, seed, n_rounds_cv=500, learning_rate=0.02, store=True, verbose=False)
    
    y_qlow, y_qup = range_modification(results["boosted_lower"].flatten(), results["boosted_upper"].flatten(), 1, 5)
    intervals = [
    [(low, high)] for low, high in zip(y_qlow, y_qup)
    ]

    df = pd.DataFrame({
        'low':    [iv[0][0] for iv in intervals],
        'up':     [iv[0][1] for iv in intervals],
        'y_test': y_test
    })

    df.to_csv(f'Boosted_{type}_{dataset}_{dimension}_{seed}.csv', index=False)

    adjusted_intervals = [
    [
        (
            next((num for num in [1, 2, 3, 4, 5] if abs(low - num) < 0.1), low),
            next((num for num in [1, 2, 3, 4, 5] if abs(high - num) < 0.1), high)
        )
        for low, high in sample_intervals
    ]
    for sample_intervals in intervals]

    intervals = adjusted_intervals

    in_interval = [
        any(low <= true_value <= high for low, high in sample_intervals)
        for sample_intervals, true_value in zip(intervals, y_test)
    ]

    coverage_rate = np.mean(in_interval)
    average_width = np.mean([high - low for sample_intervals in intervals for low, high in sample_intervals])  

    return average_width, coverage_rate

def calculate_statistics(X, y, num_runs=30, seed_start=1, dataset='summeval', dimension='consistency'):
    from tqdm import tqdm
    time_lcp = []
    memory_lcp = []
    time_cqr = []
    memory_cqr = []
    for i in tqdm(range(num_runs), desc="Running experiments"):
        seed = seed_start + i 
        tracemalloc.start()
        start = time.perf_counter()
        average_width, coverage_rate = run_experiment(X, y, seed, 'lcp', dataset, dimension)
        end = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        time_lcp.append(end - start)
        memory_lcp.append(peak/1024/1024)

        tracemalloc.start()
        start = time.perf_counter()
        average_width, coverage_rate = run_experiment(X, y, seed, 'cqr', dataset, dimension)
        end = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        time_cqr.append(end - start)
        memory_cqr.append(peak/1024/1024)

    mean_time_lcp = np.mean(time_lcp)
    std_time_lcp = np.std(time_lcp)
    mean_memory_lcp = np.mean(memory_lcp)
    std_memory_lcp = np.std(memory_lcp)
    mean_time_cqr = np.mean(time_cqr)
    std_time_cqr = np.std(time_cqr)
    mean_memory_cqr = np.mean(memory_cqr)
    std_memory_cqr = np.std(memory_cqr)

    return mean_time_lcp, std_time_lcp, mean_memory_lcp, std_memory_lcp, mean_time_cqr, std_time_cqr, mean_memory_cqr, std_memory_cqr


In [16]:
import os
import pandas as pd

folder_path = f'../model_logits/qwen/'
for dimension in ["cosmos", "drop", "esnli", "gsm8k"]:
        file_path = os.path.join(folder_path, f"SocREval_{dimension}_logits.csv")
        df = pd.read_csv(file_path)
        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]
        mean_time_lcp, std_time_lcp, mean_memory_lcp, std_memory_lcp, mean_time_cqr, std_time_cqr, mean_memory_cqr, std_memory_cqr =  calculate_statistics(X, y, num_runs=10, seed_start=1, dimension=dimension, dataset='SocREval')



Running experiments:   0%|          | 0/10 [00:00<?, ?it/s]

Fitting RF model with 97 observations...
local_cov: 1.0
local_len: 4.358804123711344
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  206
local_cov: 1.0
local_len: 4.358804123711344
boosted_cov: 0.7346938775510204
boosted_len: 3.685181300118857
Fitting QR model with 97 observations...
Fitting complete.
Making predictions using calibration data... done.
Making predictions using test data... done.
Computing score correction: -0.083
Making predictions using test data... done.
cqr_cov: 0.8367346938775511
cqr_len: 3.1590798
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.


Running experiments:  10%|█         | 1/10 [02:37<23:40, 157.83s/it]

Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  0
cqr_cov: 0.8367346938775511
cqr_len: 3.1590798
boosted_cov: 0.8367346938775511
boosted_len: 3.1590798
Fitting RF model with 97 observations...
local_cov: 1.0
local_len: 4.410721649484529
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  498
local_cov: 1.0
local_len: 4.410721649484529
boosted_cov: 0.673469387755102
boosted_len: 3.5748661326980864
Fitting QR model with 97 observations...
Fitting complete.
Making predictions using calibration data... done.
Making predictions using test data... done.
Computing score correction: 0.569
Making predictions using test data... done.
cqr_cov: 0.9591836734693877
cqr_len: 5.290195
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.


Running experiments:  20%|██        | 2/10 [05:26<21:51, 163.97s/it]

...

cqr_cov: 0.92
cqr_len: 4.328333
boosted_cov: 0.91
boosted_len: 2.950301
Fitting RF model with 100 observations...
local_cov: 1.0
local_len: 4.331600000000006
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  211
local_cov: 1.0
local_len: 4.331600000000006
boosted_cov: 0.79
boosted_len: 3.665541519085757
Fitting QR model with 100 observations...
Fitting complete.
Making predictions using calibration data... done.
Making predictions using test data... done.
Computing score correction: -0.149
Making predictions using test data... done.
cqr_cov: 0.79
cqr_len: 3.3488965
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  194


Running experiments:  90%|█████████ | 9/10 [26:42<02:56, 176.71s/it]

cqr_cov: 0.79
cqr_len: 3.3488965
boosted_cov: 0.76
boosted_len: 2.7955616
Fitting RF model with 100 observations...
local_cov: 1.0
local_len: 4.80394
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  436
local_cov: 1.0
local_len: 4.80394
boosted_cov: 0.81
boosted_len: 3.4010199058768142
Fitting QR model with 100 observations...
Fitting complete.
Making predictions using calibration data... done.
Making predictions using test data... done.
Computing score correction: -0.138
Making predictions using test data... done.
cqr_cov: 0.78
cqr_len: 2.9403305
Starting cross-validation for optimal number of boosting rounds...
Fold 0 completed.
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Cross Validation completed.
Optimal boosting rounds:  349


Running experiments: 100%|██████████| 10/10 [29:46<00:00, 178.70s/it]

cqr_cov: 0.78
cqr_len: 2.9403305
boosted_cov: 0.75
boosted_len: 2.673624


In [17]:
mean_time_lcp, std_time_lcp, mean_memory_lcp, std_memory_lcp, mean_time_cqr, std_time_cqr, mean_memory_cqr, std_memory_cqr

(87.26619162999559,
 2.573793670426636,
 2.0546069145202637,
 0.005519548689868888,
 91.429584169999,
 3.2352666031439288,
 0.8886855125427247,
 0.05398571061807915)